# Like Vegetation Trend Series in the Ruko Conservancy

* **Products used:** 
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)


## Background
Phenology is the study of plant and animal life cycles in the context of the seasons.
It can be useful in understanding the life cycle trends of crops and how the growing seasons are affected by changes in climate.
For more information, see the [USGS page on deriving phenology](https://www.usgs.gov/land-resources/eros/phenology/science/deriving-phenological-metrics-ndvi?qt-science_center_objects=0#qt-science_center_objects)


## Description

This notebook will produce annual, smoothed, **one-dimensional (zonal mean across a region)** time-series of a remote sensing vegetation indice, such as NDVI or EVI.  In addition, basic phenology statistics are calculated, exported to disk as csv files, and annotated on a plot.

A number of steps are required to produce the desired outputs:

1. Load satellite data for a region specified by an vector file (shapefile or geojson)
2. Buffer the cloud masking layer to better mask clouds in the data (Sentinel-2 cloud mask is quite poor)
3. Further prepare the data for analysis by removing bad values (infs), masking surafce water, and removing outliers in the vegetation index.
4. Calculate a zonal mean across the study region (collapse the x and y dimension by taking the mean across all pixels for each time-step).
5. Interpolate and smooth the time-series to ensure a consistent dataset with all gaps and noise removed.
6. Calculate phenology statistics, report the results, save the results to disk, and generate an annotated plot.

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Load key Python packages and supporting functions for the analysis.

In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
import geopandas as gpd
import matplotlib.pyplot as plt
from datacube.utils import geometry
import geopandas as gpd

import sys

from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.plotting import map_shapefile
from deafrica_tools import temporal as ts

from deafrica_tools.classification import HiddenPrints

from datacube.utils.aws import configure_s3_access
configure_s3_access(aws_unsigned=True, cloud_defaults=True)


/env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
/env/lib/python3.8/site-packages/datacube/storage/masking.py:7: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  warnings.warn("datacube.storage.masking has moved to datacube.utils.masking",


## Set up a Dask cluster

Dask can be used to better manage memory use down and conduct the analysis in parallel. For an introduction to using Dask with Digital Earth Africa, see the Dask notebook.

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:40969 Dashboard: /user/nanaboamah/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


### Analysis parameters

The following cell sets important parameters for the analysis:

* `veg_proxy`: Band index to use as a proxy for vegetation health e.g. `'NDVI'` or `'EVI'`
* `product`: The satellite product to load. Either Sentinel-2: `'s2_l2a'`, or Landsat-8: `'ls8_cl2'`
* `shapefile`: The path to the vector file delineating the analysis region. Can be a shapefile or a geojson
* `time_range`: The year range to analyse (e.g. `('2017-01-01', '2019-12-30')`).
* `min_gooddata`: the fraction of good data (not cloudy) a scene must have before it is returned as a dataset 
* `resolution`: The pixel resolution, in metres, of the returned dataset
* `dask_chunks`: The size, in number of pixel, for the dask chunks on each dimension.

In [3]:
veg_proxy = 'MSAVI'

product = 's2_l2a'

shapefile='data/ccy.geojson'

time_range = ('2018-01-01','2018-12-31')

min_gooddata = 0.50

resolution = (-50,50)

dask_chunks = {'x':1500, 'y':1500}


### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [4]:
dc = datacube.Datacube(app='Live_Vegetation_phenology')

## View the region of interest
The next cell will display the selected area on an web map.

In [5]:
#First open the shapefile using geopandas
gdf = gpd.read_file(shapefile)

In [6]:
map_shapefile(gdf, attribute='ConsrvName')

Label(value='')

Map(center=[1.158755580035329, 37.752218372858266], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Load cloud-masked Sentinel-2 data

The first step is to load Sentinel-2 data for the specified area of interest and time range. 
The `load_ard` function is used here to load data that has been masked for cloud, shadow and quality filters, making it ready for analysis.

The cell directly below will create a query object using the first geometry in the shapefile, along with the parameters we defined in the Analysis Parameters section above.

In [7]:
# Create a reusable query
geom = geometry.Geometry(geom=gdf.iloc[0].geometry, crs=gdf.crs)

query = {
    "geopolygon": geom,
    'time': (time_range),
    'resolution': resolution,
    'output_crs': 'epsg:6933',
    'group_by':'solar_day'
}


Load available data from S2:

In [8]:
ds = load_ard(
    dc=dc,
    products=['s2_l2a'],
    dask_chunks=dask_chunks,
    min_gooddata=min_gooddata,
    measurements =['red','blue','nir','green','swir_1','SCL'],
    **query,
)

print(ds)

/home/jovyan/Tools/deafrica_tools/datahandling.py:260: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn(


Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 60 out of 71 time steps with at least 50.0% good quality pixels
Applying pixel quality/cloud mask
Returning 60 time steps as a dask array
<xarray.Dataset>
Dimensions:      (time: 60, x: 3455, y: 4592)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-04T07:54:25 ... 2018-12-30T07:...
  * y            (y) float64 2.626e+05 2.625e+05 ... 3.308e+04 3.302e+04
  * x            (x) float64 3.556e+06 3.556e+06 ... 3.729e+06 3.729e+06
    spatial_ref  int32 6933
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(1, 1500, 1500), meta=np.ndarray>
    blue         (time, y, x) float32 dask.array<chunksize=(1, 1500, 1500), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 1500, 1500), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 1500, 1500), meta=np.ndarray>
    swir

## Cloud Buffering

The cloud masking data for Sentinel-2 is less than perfect, and missed cloud in the data greatly impacts vegetation  calculations. Below we will buffer the cloud-masking bands in an attempt to improve the masking of poor quality data.

In [9]:
import odc.algo

#Extract boolean mask
mask = odc.algo.enum_to_bool(ds.SCL, 
                             categories=['cloud shadows', 'cloud medium probability',
                                         'cloud high probability', 'thin cirrus'])
                             
# Close mask to remove small holes in cloud, open mask to 
# remove narrow false positive cloud, then dilate
mask = odc.algo.binary_closing(mask, 2)
#mask_cleaned = odc.algo.mask_cleanup(mask, r=(2, 10))

# Add new mask as nodata pixels
ds = odc.algo.erase_bad(ds, mask)


## Mask the satellite data with shape

In [10]:
#create mask
mask = xr_rasterize(gdf,ds)

#mask data
ds = ds.where(mask)

#remove SCL since we don't need it anymore
ds = ds.drop('SCL')

#convert to float 32 to conserve memory
ds=ds.astype(np.float32)

## Calculate vegetation and water indices

In [11]:
#Multiply red and nir band by 100
ds['red'] = ds['red']  * 100
ds['nir'] = ds['nir'] * 100

In [12]:
def field_MSAVI2(ds):
    return (ds + 0.0018) / 0.877 

def live_veg_frac_cover(ds):
    ds_live = (103.89 * ds) * 2.55
    return ds_live.astype(np.uint8)

In [13]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
ds = calculate_indices(ds, index=[veg_proxy, 'MNDWI'], collection='s2')

In [14]:
ds['field_MSAVI2'] = field_MSAVI2(ds['MSAVI'])

In [15]:
ds['live_veg_frac_cover'] = live_veg_frac_cover(ds['field_MSAVI2'])

In [16]:
#drop bands that are no longer needed (save memory)
ds = ds.drop(['red','green','nir','blue','swir_1', 'field_MSAVI2'])

## Prepare data for analysis

Remove any NaN or infinite values, mask water, remove any outliers in the vegetation index.  We then reduce the data to a 1D timeseries by calculating the mean across the x and y dimensions.  

We will also 'compute' the data on the dask cluster to speed up calculations later on. This step will take 5-10mins to run since we are now computing everything that came before.

In [17]:
veg_proxy = 'live_veg_frac_cover'
# remove any infinite values
ds = ds.where(xr.ufuncs.isfinite(ds))

# mask water
ds = ds.where(ds.MNDWI < 0)

ds[veg_proxy] =  ds[veg_proxy] / 2.55


In [ ]:
ds[veg_proxy] = ds[veg_proxy].compute()

In [ ]:
ds[veg_proxy] = xr.where(ds[veg_proxy] < 0, np.nan, ds[veg_proxy])

In [ ]:
# create 1D line plots
veg = ds[veg_proxy].mean(['x', 'y'])

## Smooth and interpolate time series

Due to many factors (e.g. cloud obscuring the region, missed cloud cover in the SCL layer) the data will be gappy and noisy. Here, we will smooth and interpolate the data to ensure we working with a consistent time-series.

To do this we take two steps:

1. Resample the data to fortnightly time-steps using the fortnightly median
2. Calculate a rolling mean with a window of 4 steps

In [ ]:
resample_period='4W'
window=4

veg_smooth=veg.resample(time=resample_period, label='left', loffset='1W').median().rolling(time=window, min_periods=1).mean()

## Plot the entire time-series

In [ ]:
veg_smooth.plot.line('b-^', figsize=(15,5))


plt.title(veg_proxy+' time-series');
plt.savefig(f'results/yearly_fractional_cover_{time_range}_plot.png');

In [ ]:
rf_ds = dc.load(product='rainfall_chirps_monthly', **query)

In [ ]:
# plot
fig, ax1 = plt.subplots(figsize=(15,8))

# plt.subplot(2,1,1)
rf_ds['rainfall'].sum(['x','y']).plot(marker='^', markersize=4, linewidth=1, ax=ax1, linestyle='dashed', 
                                    label=' Rainfall');

plt.ylabel('%s (%s)'%('Total Precipitation', rf_ds['rainfall'].attrs['units']));
plt.title('')

ax2 = ax1.twinx()

veg_smooth.plot(color='green', marker='^', markersize=4, linewidth=1, ax=ax2, 
                         label='Live Vegetation Cover')
plt.title('')
plt.ylabel('Live Vegatation Cover(%)', color='green')
plt.yticks(color='green')

fig.legend(loc='upper left', bbox_to_anchor=(0.05, 0.93))
fig.suptitle(f'Live Vegetation Cover compared to rainfall (CHIRPS) over time from {time_range[0]} to {time_range[1]}')
fig.tight_layout()

plt.savefig(f'results/yearly_fractional_cover_rainfall_{time_range}_plot.png');

fig.show()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')